# Packages

In [ ]:
import os
import json
import logging
from itertools import combinations

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona
import shapely
from shapely.geometry import Point
from shapely.ops import unary_union, nearest_points

import networkx as nx
import osmnx as ox

import scipy
from scipy.spatial import cKDTree

import matplotlib.pyplot as plt

import folium
from folium import GeoJson, LayerControl
from branca.colormap import linear

# Functions

In [ ]:
def tag_flooded_roads(edges, nodes, flood_zones, name):
    output_path = cut_roads_files[name]
    graphml_path = safe_roads_files[name]

    #if os.path.exists(output_path) and layer in fiona.listlayers(output_path):
    if os.path.exists(output_path):
        print(f"Loading {name} from {output_path}")
        G_safe = ox.load_graphml(graphml_path)
        edges = gpd.read_file(output_path, layer=name)
        
    else:
        print(f"Tagging and saving {name} to {output_path}")

        bounds = edges.total_bounds
        flood_subset = flood_zones.cx[bounds[0]:bounds[2], bounds[1]:bounds[3]]
        flood_geoms = flood_subset.geometry

        edges = edges.copy()
        edges["in_flood_zone"] = edges.geometry.apply(lambda geom: flood_geoms.intersects(geom).any())

        edges.to_file(output_path, layer=name, driver="GPKG")

    if os.path.exists(graphml_path):
        print("Pruned graph already exists")
    else:    
        safe_edges = edges[~edges["in_flood_zone"]].copy()
        print("Rebuilding pruned graph...")
        G_safe = ox.graph_from_gdfs(nodes, safe_edges)
        ox.save_graphml(G_safe, filepath=graphml_path)
        print(f"Saved pruned graph to {graphml_path}")

    return edges, G_safe

In [ ]:
def clip_flood_zone(return_crs, name, clip_geom):
    output_path = zone_output_files[name]
    input_path = zone_input_files[name]
    
    if os.path.exists(output_path):
        print(f"Loading {name} from {output_path}")
        clipped = gpd.read_file(output_path, layer=name).to_crs(return_crs)
    else:
        print(f"Clipping and saving {name} from {output_path}" )
        flood = gpd.read_file(input_path).to_crs(return_crs)
        clipped = gpd.clip(flood, clip_geom)
        clipped.to_file(output_path, layer=name, driver="GPKG")

    return clipped

In [ ]:
def make_geojson_safe(gdf):
    gdf = gdf.copy()
    dt_cols = gdf.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns
    gdf[dt_cols] = gdf[dt_cols].astype(str)
    for col in gdf.columns:
        if col != "geometry" and not pd.api.types.is_scalar(gdf[col].iloc[0]):
            gdf.drop(columns=[col], inplace=True)
    return gdf

In [ ]:
def add_flood_zone_layer(name, m):
    gdf=flood_zones_var[name]
    color=color_palette[name]
    if gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

    gdf_serializable = make_geojson_safe(gdf)

    style_function = lambda x: {
        'fillColor': color,
        'color': color,
        'weight': 1,
        'fillOpacity': 0.4
    }

    geojson = folium.GeoJson(
        data=gdf_serializable,
        name=f"Flood {name}",
        style_function=style_function,
        show=False
    )
    geojson.add_to(m)

In [ ]:
def add_roads_layer(name, m, flood):
    if flood:
        roads = flood_edges_var[name].copy()
        roads = roads[roads["in_flood_zone"] == flood]
    else:
        roads = edges.copy()

    roads = roads.to_crs(epsg=4326)
    roads = make_geojson_safe(roads)

    style_function = lambda x: {
        'color': color_palette[name],
        'weight': 2,
        'opacity': 0.6
    }

    if flood == True:
        geojson = folium.GeoJson(
            roads,
            name=f"Flooded Roads {name}",
            style_function=style_function,
            show=False
        )
    else:
            geojson = folium.GeoJson(
            roads,
            name=f"All roads",
            style_function=style_function,
            show=False
        )
    
    geojson.add_to(m)

In [ ]:
def parse_depth_range(val):
    if pd.isna(val):
        return None

    val = val.strip()

    if val.startswith('Below'):
        return float(val[5:].strip()) / 2

    if val.startswith('>'):
        return float(val[1:].strip())  # You may want to cap it

    if '-' in val:
        parts = val.split('-')
        try:
            low = float(parts[0].strip())
            high = float(parts[1].strip())
            return (low + high) / 2
        except:
            return None

    try:
        return float(val)
    except:
        return None

In [ ]:
def shortest_path_travel_time(G, source, target, node_to_muni):
    try:
        path = nx.shortest_path(G, source=source, target=target, weight='travel_time')
        total_travel_time = nx.shortest_path_length(G, source=source, target=target, weight='travel_time')
        key = f"{node_to_muni[source]}__{node_to_muni[target]}"

    except (nx.NetworkXNoPath, nx.NodeNotFound):
        #print(f"No path between {source} and {target} ({node_to_muni[source]} - {node_to_muni[target]})")
        path = ""
        total_travel_time = 0
        key = f"{node_to_muni[source]}__{node_to_muni[target]}"
    return key, path, total_travel_time
    

In [ ]:
def calculate_shortest_paths(G, output_dir, path_filename):
    os.makedirs(output_dir, exist_ok=True)
    municipal_nodes = [node for node, data in G.nodes(data=True) if data.get("municipality")]
    municipal_nodes = sorted(municipal_nodes)
    node_to_muni = {node: G.nodes[node]['municipality'] for node in municipal_nodes}
    path_path = os.path.join(output_dir, path_filename)

    if os.path.exists(path_path):
        logging.info(f"Shortest paths already calculated at {path_path}")
        with open(path_path, "r") as f:
            paths_dict = json.load(f)
        travel_times = [v["total_travel_time"] for v in paths_dict.values()]
        return travel_times

    logging.info(f"Calculating shortest paths between all municipalities for graph...")

    paths_dict = {}
    travel_times = []

    for source, target in combinations(municipal_nodes, 2):
        key, path, travel_time = shortest_path_travel_time(G, source, target, node_to_muni)
        paths_dict[key] = {
            "nodes": path,
            "total_travel_time": travel_time
        }
        travel_times.append(travel_time)

    with open(path_path, "w") as f:
        json.dump(paths_dict, f, indent=2)

    logging.info(f"Saved shortest paths to {path_path}")

    return travel_times

In [ ]:
def compute_individual_risk_factor(T_P,T_NP):
    if T_P==0:
        return 1
    else:
        return 1-(T_NP/T_P)

In [ ]:
def compute_risk_factor(T_P_list, T_NP_list):
    R=0
    for i in range(len(T_P_list)):
        R+= compute_individual_risk_factor(T_P_list[i],T_NP_list[i])
    R/=(len(T_P_list))
    return R

In [ ]:
def flood_depth_zones(name):
    layer="depth_val"
    input_path = depth_input_files[name]
    output_path=depth_output_files[name]
    if os.path.exists(output_path):
        print(f"Loading {layer} from {output_path}")
        depth=gpd.read_file(output_path, layer=layer)
    else:
        print(f"Saving {layer} to {output_path}")
        depth = gpd.read_file(input_path)
        depth["depth_val"] = depth["value"].apply(parse_depth_range)
        depth.to_file(output_path, layer=layer, driver="GPKG")
        print(f"Saved processed {layer} in {output_path}")
    return depth

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# Street data

In [ ]:
output_dir = "processed_files"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
ox.settings.use_cache = True
ox.settings.log_console = False

In [ ]:
polygon_path = os.path.join(output_dir, "study_area.geojson")
graph_path = os.path.join(output_dir, "road_graph.graphml")

valencia_municipalities = [
    "Alaquàs", "Albal", "Albalat de la Ribera", "Alberic", "Alborache", "Alcàsser", "l'Alcúdia",
    "Aldaia", "Alfafar", "Alfarb", "Algemesí", "Alginet", "Almussafes", "Alzira",
    "Benetússer", "Benicull de Xúquer", "Benifaió", "Beniparrell", "Benimodo", "Bétera",
    "Bugarra", "Buñol", "Calles", "Camporrobles", "Carcaixent", "Carlet", "Catadau", "Catarroja",
    "Caudete de las Fuentes", "Chera", "Cheste", "Chiva", "Chulilla", "Corbera", "Cullera",
    "Dos Aguas", "Favara", "Fortaleny", "Fuenterrobles", "Gestalgar", "Godelleta", "Guadassuar",
    "l'Ènova", "Llaurí", "Llombai", "Llíria", "Llocnou de la Corona", "Loriguilla", "Macastre",
    "Manises", "Manuel", "Massanassa", "Millares", "Mislata", "Montroi", "Montserrat", "Paiporta",
    "Paterna", "Pedralba", "Picanya", "Picassent", "Polinyà de Xúquer", "La Pobla Llarga",
    "Quart de Poblet", "Rafelguaraf", "Real", "Requena", "Riba-roja de Túria", "Riola", "Sedaví",
    "Senyera", "Siete Aguas", "Silla", "Sinarcas", "Sollana", "Sot de Chera", "Sueca",
    "Tavernes de la Valldigna", "Torrent", "Tous", "Turís", "Utiel", "València", "Vilamarxant", "Xirivella",
    "Yátova"
]

urban_center={
    "Alaquàs": (39.457119, -0.460822), 
    "Albal": (39.397347, -0.413243), 
    "Albalat de la Ribera": (39.201133, -0.386659),
    "Alberic": (39.116783, -0.517571),
    "Alborache": (39.391716, -0.771191),
    "Alcàsser": (39.369810, -0.445176),
    "l'Alcúdia": (39.853110, 3.121137),
    "Aldaia": (39.464957, -0.460764),
    "Alfafar": (39.422773, -0.390834), 
    "Alfarb": (39.276849, -0.560529),
    "Algemesí": (39.188743, -0.436927),
    "Alginet": (39.261819, -0.469791),
    "Almussafes": (39.292656, -0.413260),
    "Alzira": (39.152038, -0.441074),
    "Benetússer": (39.423801, -0.397785),
    "Benicull de Xúquer": (39.185063, -0.382524), 
    "Benifaió": (39.285003, -0.426889), 
    "Beniparrell": (39.382235, -0.412277), 
    "Benimodo": (39.213015, -0.528357), 
    "Bétera": (39.591643, -0.462355),
    "Bugarra": (39.608415, -0.775958), 
    "Buñol": (39.418217, -0.790672), 
    "Calles": (39.725371, -0.973952), 
    "Camporrobles": (39.647167, -1.399476), 
    "Carcaixent": (39.122600, -0.450718), 
    "Carlet": (39.225881, -0.519809), 
    "Catadau": (39.275363, -0.569740), 
    "Catarroja": (39.403048, -0.404021),
    "Caudete de las Fuentes": (39.558958, -1.278492), 
    "Chera": (39.593442, -0.973241), 
    "Cheste": (39.494909, -0.684261), 
    "Chiva": (39.471469, -0.717100), 
    "Chulilla": (39.656217, -0.891905), 
    "Corbera": (39.158264, -0.355378), 
    "Cullera": (39.165250, -0.253612),
    "Dos Aguas": (39.288825, -0.800238), 
    "Favara": (39.127275, -0.291808), 
    "Fortaleny": (39.183833, -0.313715), 
    "Fuenterrobles": (39.584682, -1.364031), 
    "Gestalgar": (39.604207, -0.834346), 
    "Godelleta": (39.422040, -0.686434), 
    "Guadassuar": (39.185852, -0.478023),
    "l'Ènova": (39.045134, -0.480790), 
    "Llaurí": (39.147102, -0.330235), 
    "Llombai": (39.282414, -0.572366), 
    "Llíria": (39.624719, -0.595006), 
    "Llocnou de la Corona": (39.420446, -0.382216), 
    "Loriguilla": (39.489698, -0.571964), 
    "Macastre": (39.381938, -0.785287),
    "Manises": (39.493345, -0.457466), 
    "Manuel": (39.052760, -0.494055), 
    "Massanassa": (39.411460, -0.397986), 
    "Millares": (39.238985, -0.773203), 
    "Mislata": (39.475218, -0.417833), 
    "Montroi": (39.341742, -0.613689), 
    "Montserrat": (39.358012, -0.603212), 
    "Paiporta": (39.429588, -0.417467),
    "Paterna": (39.500663, -0.439682), 
    "Pedralba": (39.604864, -0.726535), 
    "Picanya": (39.435146, -0.433490), 
    "Picassent": (39.362746, -0.458078), 
    "Polinyà de Xúquer": (39.196150, -0.369461), 
    "La Pobla Llarga": (39.085916, -0.475557),
    "Quart de Poblet": (39.483003, -0.442288), 
    "Rafelguaraf": (39.050797, -0.455126), 
    "Real": (39.335088, -0.609125), 
    "Requena": (39.487037, -1.098087), 
    "Riba-roja de Túria": (39.546926, -0.566891), 
    "Riola": (39.195261, -0.334682), 
    "Sedaví": (39.425005, -0.385783),
    "Senyera": (39.063613, -0.510467), 
    "Siete Aguas": (39.471633, -0.915872), 
    "Silla": (39.362679, -0.412147), 
    "Sinarcas": (39.733258, -1.229035), 
    "Sollana": (39.278009, -0.381457), 
    "Sot de Chera": (39.620816, -0.909344), 
    "Sueca": (39.202640, -0.310637),
    "Tavernes de la Valldigna": (39.071834, -0.267740), 
    "Torrent": (39.436931, -0.465889), 
    "Tous": (39.138561, -0.586636), 
    "Turís": (39.389834, -0.711107), 
    "Utiel": (39.566851, -1.206009), 
    "València": (39.469844, -0.376852), 
    "Vilamarxant": (39.567855, -0.622488), 
    "Xirivella": (39.463360, -0.428399),
    "Mira": (39.720949, -1.439225)
}  

In [ ]:
if os.path.exists(polygon_path):
    logging.info("Loading saved study area polygon...")
    study_area = gpd.read_file(polygon_path)
else:
    logging.info("Downloading polygons for municipalities...")
    polygons = []

    for municipality in valencia_municipalities:
        try:
            place_name = f"{municipality}, Valencia, Spain"
            gdf = ox.geocode_to_gdf(place_name)

            if gdf.crs != "EPSG:4326":
                gdf = gdf.to_crs("EPSG:4326")

            polygons.append(gdf)
        except Exception as e:
            logging.warning(f"Error retrieving {municipality}: {e}", exc_info=True)

    # Add Mira from Cuenca (just in case)
    try:
        gdf = ox.geocode_to_gdf("Mira, Cuenca, Spain")
        if gdf.crs != "EPSG:4326":
            gdf = gdf.to_crs("EPSG:4326")
        polygons.append(gdf)
    except Exception as e:
        logging.warning(f"Error retrieving Mira: {e}", exc_info=True)

    study_area = gpd.GeoDataFrame(pd.concat(polygons, ignore_index=True), crs="EPSG:4326")
    study_area = study_area[study_area.geometry.notnull()]
    study_area.to_file(polygon_path, driver="GeoJSON")

In [ ]:
study_area = study_area[study_area.geometry.type.isin(["Polygon", "MultiPolygon"])]
polygon = unary_union(study_area.geometry)
logging.info("Polygon geometry union complete.")

In [ ]:
if os.path.exists(graph_path):
    logging.info("Loading saved road network graph...")
    G = ox.load_graphml(graph_path)
    
else:
    logging.info("Downloading road network...")

    G = ox.graph_from_polygon(
        polygon,
        network_type="drive",
        simplify=True,
        retain_all=False,
        truncate_by_edge=True
    )

    logging.info("Calculating travel times...")
    for u, v, k, data in G.edges(keys=True, data=True):
        if "length" in data:
            speed = None

            if "maxspeed" in data:
                maxspeed = data["maxspeed"]
                if isinstance(maxspeed, list):
                    speed = maxspeed[0]
                else:
                    speed = maxspeed

                try:
                    speed = float(str(maxspeed).split()[0])  # Handle "50 km/h" etc.
                except ValueError:
                    speed = None  # Fallback to highway-based speed below

            if speed is None:
                highway = data.get("highway", "")
                if isinstance(highway, list):
                    highway = highway[0]
                speed = {
                    "motorway": 120,
                    "motorway_link": 60,
                    "trunk": 100,
                    "primary": 80,
                    "secondary": 60,
                    "tertiary": 50,
                    "residential": 30,
                    "living_street": 10,
                    "unclassified": 40,
                    "service": 20
                }.get(highway, 50)  # fallback 50 km/h

            surface = data.get("surface", "").lower()
            surface_speed_factor = {
                "paved": 1.0,
                "asphalt": 1.0,
                "concrete": 1.0,
                "cobblestone": 0.8,
                "gravel": 0.7,
                "dirt": 0.6,
                "ground": 0.6,
                "sand": 0.5,
                "unpaved": 0.7,
                "compacted": 0.85,
                "fine_gravel": 0.9
            }
            for key, factor in surface_speed_factor.items():
                if key in surface:
                    speed *= factor
                    break 

            speed_mps = speed * 1000 / 3600
            data["travel_time"] = data["length"] / speed_mps
            data["travel_time"] += 5 # Turn penalty approximation

    ox.save_graphml(G, filepath=graph_path)
    logging.info("Graph saved.")

nodes, edges = ox.graph_to_gdfs(G)
logging.info("Converted graph to GeoDataFrames.")

In [ ]:
node_coords = np.array([(geom.y, geom.x) for geom in nodes.geometry])
kdtree = cKDTree(node_coords)

if 'municipality' not in nodes.columns:
    nodes['municipality'] = ""

for name, (lat, lon) in urban_center.items():
    try:
        # Query closest node index
        _, idx = kdtree.query([lat, lon], k=1)
        nearest_node_idx = nodes.index[idx]
        nodes.at[nearest_node_idx, 'municipality'] = name
    except Exception as e:
        logging.warning(f"Error assigning municipality {name}: {e}")


for node_id, row in nodes.iterrows():
    G.nodes[node_id]['municipality'] = row['municipality']

ox.save_graphml(G, graph_path)
logging.info(f"Updated graph saved to: {graph_path}")        

In [ ]:
from scipy.spatial import cKDTree

if 'municipality' not in nodes.columns:
    logging.info("Adding 'municipality' field to nodes...")

    nodes['municipality'] = ""

    node_coords = np.array([(geom.y, geom.x) for geom in nodes.geometry])
    kdtree = cKDTree(node_coords)

    for _, row in study_area.iterrows():
        name = row.get("name") or row.get("display_name") or "unknown"
        geom = row.geometry

        if not geom or not geom.is_valid or name == "Favara":
            continue

        try:
            muni_graph = ox.graph_from_polygon(geom, network_type='drive', simplify=True)
            center_node = ox.distance.nearest_nodes(muni_graph, X=geom.centroid.x, Y=geom.centroid.y)
            center_point = Point((muni_graph.nodes[center_node]['x'], muni_graph.nodes[center_node]['y']))
            _, idx = kdtree.query([center_point.y, center_point.x], k=1)
            nearest_node_idx = nodes.index[idx]
            nodes.at[nearest_node_idx, 'municipality'] = name

        except Exception as e:
            logging.warning(f"Could not process {name}: {e}")
            continue

    for node_id, row in nodes.iterrows():
        G.nodes[node_id]['municipality'] = row['municipality']

    graph_path = os.path.join(output_dir, "road_graph.graphml")
    ox.save_graphml(G, graph_path)
    logging.info(f"Updated graph saved to: {graph_path}")

else:
    logging.info("'municipality' field already exists in nodes.")

In [ ]:
rail=False
if rail:
    graph_path_rail = os.path.join(output_dir, "study_area_rail.graphml")
    if os.path.exists(graph_path_rail):
        print("Loading saved rail network graph...")
        G_rail = ox.load_graphml(graph_path_rail)
    else:
        print("Downloading rail network graph...")
        rail_filter = '["railway"~"rail|light_rail|subway|tram"]'
        G_rail = ox.graph_from_polygon(polygon, custom_filter=rail_filter, network_type="all")
        ox.save_graphml(G_rail, filepath=graph_path_rail)

    nodes_rail, edges_rail = ox.graph_to_gdfs(G_rail)
    del G_rail

# Layers

In [ ]:
input_dir = "source_files"
output_dir = "processed_files"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
zone_input_files = {
    "10 yr": f"{input_dir}/laminaspb-q10/Q10_2Ciclo_PB_20241121.shp",
    "100 yr": f"{input_dir}/laminaspb-q100/Q100_2Ciclo_PB_20241121_ETRS89.shp",
    "500 yr": f"{input_dir}/laminaspb-q500/Q500_2Ciclo_PB_20241121_ETRS89.shp",
    "DANA_31_10_2024": f"{input_dir}/EMSR773_AOI01_DEL_PRODUCT_v1/EMSR773_AOI01_DEL_PRODUCT_observedEventA_v1.shp",
    "DANA_03_11_2024": f"{input_dir}/EMSR773_AOI01_DEL_MONIT01_v1/EMSR773_AOI01_DEL_MONIT01_observedEventA_v1.shp",
    "DANA_05_11_2024": f"{input_dir}/EMSR773_AOI01_DEL_MONIT02_v1/EMSR773_AOI01_DEL_MONIT02_observedEventA_v1.shp",
    "DANA_06_11_2024": f"{input_dir}/EMSR773_AOI01_DEL_MONIT03_v1/EMSR773_AOI01_DEL_MONIT03_observedEventA_v1.shp",
    "DANA_08_11_2024": f"{input_dir}/EMSR773_AOI01_DEL_MONIT04_v1/EMSR773_AOI01_DEL_MONIT04_observedEventA_v1.shp"
}

depth_input_files = {
    "DANA_31_10_2024": f"{input_dir}/EMSR773_AOI01_DEL_PRODUCT_v1/EMSR773_AOI01_DEL_PRODUCT_floodDepthA_v1.shp",
    "DANA_03_11_2024": f"{input_dir}/EMSR773_AOI01_DEL_MONIT01_v1/EMSR773_AOI01_DEL_MONIT01_floodDepthA_v1.shp",
    "DANA_05_11_2024": f"{input_dir}/EMSR773_AOI01_DEL_MONIT02_v1/EMSR773_AOI01_DEL_MONIT02_floodDepthA_v1.shp",
    "DANA_06_11_2024": f"{input_dir}/EMSR773_AOI01_DEL_MONIT03_v1/EMSR773_AOI01_DEL_MONIT03_floodDepthA_v1.shp",
    "DANA_08_11_2024": f"{input_dir}/EMSR773_AOI01_DEL_MONIT04_v1/EMSR773_AOI01_DEL_MONIT04_floodDepthA_v1.shp"
}

zone_output_files = {
    "10 yr": f"{output_dir}/zone_flood_risk_10.gpkg",
    "100 yr": f"{output_dir}/zone_flood_risk_100.gpkg",
    "500 yr": f"{output_dir}/zone_flood_risk_500.gpkg",
    "DANA_31_10_2024": f"{output_dir}/zone_DANA_31_10_2024.gpkg",
    "DANA_03_11_2024": f"{output_dir}/zone_DANA_03_11_2024.gpkg",
    "DANA_05_11_2024": f"{output_dir}/zone_DANA_05_11_2024.gpkg",
    "DANA_06_11_2024": f"{output_dir}/zone_DANA_06_11_2024.gpkg",
    "DANA_08_11_2024": f"{output_dir}/zone_DANA_08_11_2024.gpkg"
}

depth_output_files = {
    "DANA_31_10_2024": f"{output_dir}/depth_DANA_31_10_2024.gpkg",
    "DANA_03_11_2024": f"{output_dir}/depth_DANA_03_11_2024.gpkg",
    "DANA_05_11_2024": f"{output_dir}/depth_DANA_05_11_2024.gpkg",
    "DANA_06_11_2024": f"{output_dir}/depth_DANA_06_11_2024.gpkg",
    "DANA_08_11_2024": f"{output_dir}/depth_DANA_08_11_2024.gpkg"
}

cut_roads_files = {
    "10 yr": f"{output_dir}/cut_roads_flood_risk_10.graphml",
    "100 yr": f"{output_dir}/cut_roads_flood_risk_100.graphml",
    "500 yr": f"{output_dir}/cut_roads_flood_risk_500.graphml",
    "DANA_31_10_2024": f"{output_dir}/cut_roads_DANA_31_10_2024.graphml",
    "DANA_03_11_2024": f"{output_dir}/cut_roads_DANA_03_11_2024.graphml",
    "DANA_05_11_2024": f"{output_dir}/cut_roads_DANA_05_11_2024.graphml",
    "DANA_06_11_2024": f"{output_dir}/cut_roads_DANA_06_11_2024.graphml",
    "DANA_08_11_2024": f"{output_dir}/cut_roads_DANA_08_11_2024.graphml"
}

safe_roads_files = {
    "10 yr": f"{output_dir}/safe_roads_flood_risk_10.graphml",
    "100 yr": f"{output_dir}/safe_roads_flood_risk_100.graphml",
    "500 yr": f"{output_dir}/safe_roads_flood_risk_500.graphml",
    "DANA_31_10_2024": f"{output_dir}/safe_roads_DANA_31_10_2024.graphml",
    "DANA_03_11_2024": f"{output_dir}/safe_roads_DANA_03_11_2024.graphml",
    "DANA_05_11_2024": f"{output_dir}/safe_roads_DANA_05_11_2024.graphml",
    "DANA_06_11_2024": f"{output_dir}/safe_roads_DANA_06_11_2024.graphml",
    "DANA_08_11_2024": f"{output_dir}/safe_roads_DANA_08_11_2024.graphml"
}

# Flood zones

In [ ]:
layer_names=["10 yr","100 yr","500 yr","DANA_31_10_2024","DANA_03_11_2024","DANA_05_11_2024","DANA_06_11_2024","DANA_08_11_2024"]
flood_zones_var = {}
flood_edges_var = {}
flood_graph_var = {}

In [ ]:
for name in layer_names: 
    result = clip_flood_zone(edges.crs, name, polygon)
    flood_zones_var[name] = result
    result_1, result_2 = tag_flooded_roads(edges, nodes, result, name)
    flood_edges_var[name] = result_1
    flood_graph_var[name] = result_2 

In [ ]:
layer_names = ["DANA_31_10_2024","DANA_03_11_2024","DANA_05_11_2024","DANA_06_11_2024","DANA_08_11_2024"]
depth_zones = {}

In [ ]:
for name in layer_names: 
    depth = flood_depth_zones(name)
    depth_zones[name]=depth

# Navegability Analysis

In [ ]:
T_NP_list = calculate_shortest_paths(G, output_dir, path_filename="shortest_paths_travel_time.json")

In [ ]:
for name, graph in flood_graph_var.items():
    components = list(nx.weakly_connected_components(graph))
    
    multi_muni_count = sum(
        1 for component in components
        if len({graph.nodes[node]["municipality"] for node in component if graph.nodes[node]["municipality"] != ""}) > 0
    )

    print(f"{name} {multi_muni_count} weakly connected components")

    if multi_muni_count > 0:
        j = 0
        for i, component in enumerate(components):
            municipalities = {
                graph.nodes[node]["municipality"]
                for node in component
                if graph.nodes[node]["municipality"] != ""
            }
            if len(municipalities) > 0:
                j += 1
                print(f"Component {j} municipalities: {municipalities}")

In [ ]:
layer_names=["10 yr","100 yr","500 yr","DANA_31_10_2024","DANA_03_11_2024","DANA_05_11_2024","DANA_06_11_2024","DANA_08_11_2024"]
T_P_lists = {}
R={}

In [ ]:
for name in layer_names:
    T_P_lists[name] = calculate_shortest_paths(flood_graph_var[name], output_dir, "shortest_travel_time_"+name+".json")
    R[name] = compute_risk_factor(T_P_lists[name], T_NP_list)

# Plots

In [ ]:
sorted_data = dict(sorted(R.items(), key=lambda item: item[1]))
keys = list(R.keys())
values = list(R.values())

plt.figure(figsize=(8, 5))
plt.bar(keys, values, color="Blue", alpha=0.5)

plt.ylim(0, 1)
plt.ylabel('R')
plt.title('Risk Factor')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

plt.savefig('processed_files/Risk_Factor.png')

In [ ]:
color_palette = {
    # yr values — warm and spaced
    "10 yr": "#FFD700",    # Gold
    "100 yr": "#FF7F00",   # Dark Orange
    "500 yr": "#B22222",   # Firebrick 

    # DANA values — distinct, avoiding orange/red hues
    "DANA_31_10_2024": "#8A2BE2",  # Blue-Violet
    "DANA_03_11_2024": "#FF1493",  # Deep Pink
    "DANA_05_11_2024": "#00CED1",  # Dark Turquoise
    "DANA_06_11_2024": "#32CD32",  # Lime Green
    "DANA_08_11_2024": "#1E90FF",  # Dodger Blue

    # Normal condition
    "Normal Conditions": "#808080"  # Grey
}

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import gaussian_kde

# --- Prepare data ---
plot_data = []
zero_counts = {}
all_counts = {}
layer_names = ["10 yr", "100 yr", "500 yr"]

# T_P_lists scenarios
for name, times in T_P_lists.items():
    if name in layer_names:
        non_zero_times = [t / 60 for t in times if t > 0]
        all_times = [t / 60 for t in times]
        zero_counts[name] = times.count(0)
        all_counts[name] = len(times)
        plot_data.append((name, non_zero_times))

# Add normal scenario
normal_non_zero = [t / 60 for t in T_NP_list if t > 0]
normal_total = len(T_NP_list)
plot_data.append(("Normal Conditions", normal_non_zero))
all_counts["Normal Conditions"] = normal_total
zero_counts["Normal Conditions"] = T_NP_list.count(0)

# --- KDE Plot ---
fig, (ax_kde, ax_bar) = plt.subplots(1, 2, figsize=(14, 7), gridspec_kw={'width_ratios': [2.5, 1]})

x_vals = np.linspace(0, max([max(times) if times else 0 for _, times in plot_data]), 500)

for name, times in plot_data:
    if len(times) > 1:
        kde = gaussian_kde(times)
        y = kde(x_vals)
        ratio = len(times) / all_counts[name] if all_counts[name] > 0 else 0
        y_rescaled = y * ratio
        color = color_palette.get(name, 'gray')  # fallback to gray if not defined
        ax_kde.plot(x_vals, y_rescaled, label=name, color=color)

ax_kde.set_title("PDF of Travel Times")
ax_kde.set_xlabel("Travel Time (minutes)")
ax_kde.grid(True)
ax_kde.legend(loc='upper right')

# --- Reachability Bar Chart ---
scenarios = layer_names + ["Normal Conditions"]
reachable = [(all_counts[n] - zero_counts[n]) / all_counts[n] if all_counts[n] > 0 else 0 for n in scenarios]
unreachable = [1 - r for r in reachable]

bar_positions = np.arange(len(scenarios))
ax_bar.barh(bar_positions, unreachable, color='salmon', label='Unreachable')
ax_bar.barh(bar_positions, reachable, left=unreachable, color='mediumseagreen', label='Reachable')

ax_bar.set_yticks(bar_positions)
ax_bar.set_yticklabels(scenarios)
ax_bar.set_xlim(0, 1)
ax_bar.set_title("Fraction of cut routes")
ax_bar.set_xlabel("Proportion")
#ax_bar.legend(loc='lower right')
ax_bar.grid(axis='x', linestyle='--', alpha=0.5)

ax_bar.invert_yaxis()

plt.tight_layout()
plt.show()

plt.savefig('processed_files/Travel_times_yr.png')

In [ ]:

# --- Prepare data ---
plot_data = []
zero_counts = {}
all_counts = {}
layer_names = ["DANA_31_10_2024","DANA_03_11_2024","DANA_05_11_2024","DANA_06_11_2024","DANA_08_11_2024"]

# T_P_lists scenarios
for name, times in T_P_lists.items():
    if name in layer_names:
        non_zero_times = [t / 60 for t in times if t > 0]
        all_times = [t / 60 for t in times]
        zero_counts[name] = times.count(0)
        all_counts[name] = len(times)
        plot_data.append((name, non_zero_times))

# Add normal scenario
normal_non_zero = [t / 60 for t in T_NP_list if t > 0]
normal_total = len(T_NP_list)
plot_data.append(("Normal Conditions", normal_non_zero))
all_counts["Normal Conditions"] = normal_total
zero_counts["Normal Conditions"] = T_NP_list.count(0)

# --- KDE Plot ---
fig, (ax_kde, ax_bar) = plt.subplots(1, 2, figsize=(14, 7), gridspec_kw={'width_ratios': [2.5, 1]})

x_vals = np.linspace(0, max([max(times) if times else 0 for _, times in plot_data]), 500)

for name, times in plot_data:
    if len(times) > 1:
        kde = gaussian_kde(times)
        y = kde(x_vals)
        ratio = len(times) / all_counts[name] if all_counts[name] > 0 else 0
        y_rescaled = y * ratio
        color = color_palette.get(name, 'gray')  # fallback to gray if not defined
        ax_kde.plot(x_vals, y_rescaled, label=name, color=color)

ax_kde.set_title("PDF of Travel Times")
ax_kde.set_xlabel("Travel Time (minutes)")
ax_kde.grid(True)
ax_kde.legend(loc='upper right')

# --- Reachability Bar Chart ---
scenarios = layer_names + ["Normal Conditions"]
reachable = [(all_counts[n] - zero_counts[n]) / all_counts[n] if all_counts[n] > 0 else 0 for n in scenarios]
unreachable = [1 - r for r in reachable]

bar_positions = np.arange(len(scenarios))
ax_bar.barh(bar_positions, unreachable, color='salmon', label='Unreachable')
ax_bar.barh(bar_positions, reachable, left=unreachable, color='mediumseagreen', label='Reachable')

ax_bar.set_yticks(bar_positions)
ax_bar.set_yticklabels(scenarios)
ax_bar.set_xlim(0, 1)
ax_bar.set_title("Fraction of cut routes")
ax_bar.set_xlabel("Proportion")
#ax_bar.legend(loc='lower right')
ax_bar.grid(axis='x', linestyle='--', alpha=0.5)

ax_bar.invert_yaxis()

plt.tight_layout()
plt.show()

plt.savefig('processed_files/Travel_times_DANA.png')

In [ ]:
plot_data = []
zero_counts = {}

layer_names=["10 yr","100 yr","500 yr","DANA_31_10_2024","DANA_03_11_2024","DANA_05_11_2024","DANA_06_11_2024","DANA_08_11_2024"]

for name in layer_names:
    if name in T_P_lists:
        perturbed_times = T_P_lists[name]
        
        if len(perturbed_times) != len(T_NP_list):
            raise ValueError(f"Length mismatch for {name}: {len(perturbed_times)} vs {len(T_NP_list)}")

        # Calculate (perturbed - unperturbed) in minutes
        differences = [(p - u) / 60 for p, u in zip(perturbed_times, T_NP_list)]
        
        # Store (scenario name, difference) tuples
        plot_data.extend([(name, diff) for diff in differences])

# Create DataFrame for plotting
df = pd.DataFrame(plot_data, columns=["Route", "TravelTimeDifference"])

# Filter to only include positive travel time differences
df = df[df["TravelTimeDifference"] > 0]

# KDE plot of positive differences
plt.figure(figsize=(8, 6))
sns.kdeplot(data=df, x="TravelTimeDifference", hue="Route", common_norm=False, fill=False, alpha=1)
plt.axvline(0, color='black', linestyle='--', linewidth=1)
plt.title("PDF of Positive Travel Time Differences (Perturbed - Unperturbed, in Minutes)")
plt.xlabel("Travel Time Difference (minutes)")
plt.ylabel("Density")
#plt.xlim(left=0)
plt.grid(True)
plt.tight_layout()
plt.show()

# Interactive Visualizations

In [ ]:
# Set initial position
projected = study_area.to_crs(epsg=25830)
centroid_projected = projected.geometry.centroid.iloc[0]
centroid_latlon = gpd.GeoSeries([centroid_projected], crs=25830).to_crs(epsg=4326).geometry.iloc[0]
map_center = [centroid_latlon.y, centroid_latlon.x]
bounds_wgs84 = study_area.to_crs(epsg=4326).total_bounds
map_bounds = [[bounds_wgs84[1], bounds_wgs84[0]], [bounds_wgs84[3], bounds_wgs84[2]]]

## Areas at Risk and DANA Area (with roads)

In [ ]:
del m_1

In [ ]:
m_1 = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB positron", max_bounds=True)
m_1.fit_bounds(map_bounds)

In [ ]:
if os.path.exists(graph_path):
    logging.info("Loading saved road network graph...")
    G = ox.load_graphml(graph_path)
else:
    logging.info("Downloading road network...")
    G = ox.graph_from_polygon(polygon, network_type="drive", simplify=True)
    ox.save_graphml(G, filepath=graph_path)
    logging.info("Graph saved.")

nodes, edges = ox.graph_to_gdfs(G)
logging.info("Converted graph to GeoDataFrames.")

In [ ]:
# Add flood zones
add_flood_zone_layer("10 yr", m_1)
add_flood_zone_layer("100 yr", m_1)
add_flood_zone_layer("500 yr", m_1)
add_flood_zone_layer("DANA_31_10_2024", m_1)
    
# Add flooded roads (optional)
add_roads_layer("10 yr", m_1, True)
add_roads_layer("100 yr", m_1, True)
add_roads_layer("500 yr", m_1, True)
add_roads_layer("DANA_31_10_2024", m_1, True)
add_roads_layer("Normal Conditions", m_1, False)

In [ ]:
folium.LayerControl(collapsed=False).add_to(m_1)
m_1.save("processed_files/Risk_max_DANA.html")

In [ ]:
del m_1

## DANA flood depth

In [ ]:
m_2 = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB positron", max_bounds=True)
m_2.fit_bounds(map_bounds)

In [ ]:
depth=depth_zones["DANA_31_10_2024"]

In [ ]:
min_depth = depth["depth_val"].min()
max_depth = depth["depth_val"].max()
depth_colormap = linear.YlGnBu_09.scale(min_depth, max_depth)
depth_colormap.caption = 'Flood Depth (m)'

folium.GeoJson(
    depth,
    name="DANA flood depth",
    style_function=lambda feature: {
        'fillColor': depth_colormap(feature['properties']['depth_val']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=["depth_val"], aliases=["Depth (m):"])
).add_to(m_2)

depth_colormap.add_to(m_2)

In [ ]:
folium.LayerControl(collapsed=False).add_to(m_2)
m_2.save("processed_files/Max_flood_depth.html")

## DANA Flooded Area Evolution

In [ ]:
m_3 = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB positron", max_bounds=True)
m_3.fit_bounds(map_bounds)

In [ ]:
# Add flood zones
add_flood_zone_layer("DANA_31_10_2024", m_3)
add_flood_zone_layer("DANA_03_11_2024", m_3)
add_flood_zone_layer("DANA_05_11_2024", m_3)
add_flood_zone_layer("DANA_06_11_2024", m_3)
add_flood_zone_layer("DANA_08_11_2024", m_3)
    
# Add flooded roads (optional)
add_roads_layer("DANA_31_10_2024", m_3, True)
add_roads_layer("DANA_03_11_2024", m_3, True)
add_roads_layer("DANA_05_11_2024", m_3, True)
add_roads_layer("DANA_06_11_2024", m_3, True)
add_roads_layer("DANA_08_11_2024", m_3, True)
add_roads_layer("Normal Conditions", m_3, False)

In [ ]:
folium.LayerControl(collapsed=False).add_to(m_3)
m_3.save("processed_files/DANA_evolution.html")

# Old Code